In [2]:
import requests
import json
import pandas as pd
api_key = "2523f39c26f734591847049c3c35a4d6"

genre_URL = "https://api.themoviedb.org/3/genre/movie/list"
discover_URL = "https://api.themoviedb.org/3/discover/movie"
movie_id_URL = "https://api.themoviedb.org/3/search/movie"

In [3]:
class Person:
    def __init__(self, name, prefs) -> None:
        self.name = name
        self.movie_prefs = prefs
        self.fav_movies_str = ["Ad Astra", "Tenet", "Inception", "Memento"] 
        self.fav_movies = []

    def movies_to_ids(self, get_movie_ID):
        for movie in self.fav_movies_str:
            id = get_movie_ID(movie)
            if(id is not None):
                self.fav_movies.append(id)
        print(self.fav_movies)

    

In [4]:
# Return current genre dict given all genres present
# in the API. 
# DICT: Category name (str) -> ID (int)
def get_genre_dict():
    # FETCH GENRE DICTIONARY
    params = {"api_key": api_key, "language": "en-US"}
    response = requests.get(genre_URL, params=params)
    data = json.loads(response.text)
    genres = data["genres"]
    genre_dict = {}

    # Print the list of genre names and IDs
    for genre in genres:
        genre_dict[genre['name']] = genre['id']
    
    return genre_dict

def get_movie_ID(name):

    params = {"api_key": api_key, "query": "{}".format(name)}
    response = requests.get(movie_id_URL, params=params)
    movie_data = response.json()
    if(movie_data["results"] == []):
        return None
    
    movie_id = movie_data["results"][0]["id"]

    print("Movie ID:", movie_id)
    return movie_id

# Return appropriate URL for fetching movie details
def return_details_URLs(num=550):
    movie_url = "https://api.themoviedb.org/3/movie/{}".format(num)
    credits_url = "https://api.themoviedb.org/3/movie/{}/credits".format(num)

    return movie_url, credits_url

# For each preference of user, find
# correct id and append to list
def get_genre_ids(person, dict):
    ids = ""
    pref_list = []
    for elem in person.movie_prefs:
        pref_list.append(str(dict[elem]))

    my_string = ", ".join(pref_list)
    
    return my_string


In [7]:

id = get_movie_ID("Ad Astra")
if(id is not None):
    get_movie_details(id)

print("\nGENRE DICT: \n", get_genre_dict())

Movie ID: 419704
Title: Ad Astra
Genres: Science Fiction, Drama
Vote average: 6.099
Vote count: 5934
Main cast: Brad Pitt, Tommy Lee Jones, Ruth Negga, John Ortiz, Liv Tyler

GENRE DICT: 
 {'Action': 28, 'Adventure': 12, 'Animation': 16, 'Comedy': 35, 'Crime': 80, 'Documentary': 99, 'Drama': 18, 'Family': 10751, 'Fantasy': 14, 'History': 36, 'Horror': 27, 'Music': 10402, 'Mystery': 9648, 'Romance': 10749, 'Science Fiction': 878, 'TV Movie': 10770, 'Thriller': 53, 'War': 10752, 'Western': 37}


In [7]:

def get_movie_details(id=550):
    # GET data for specific movie
    params = {"api_key": api_key, "language": "en-US"}
    movie_url, credits_url = return_details_URLs(id) # Fetch correctly formatted URL for getting details for movie 

    # Get response for both credits and movie details
    response = requests.get(movie_url, params=params)
    credits_response = requests.get(credits_url, params=params)

    data = json.loads(response.text)
    data_credits = json.loads(credits_response.text)

    # Extract the relevant information
    title = data["title"]
    genre_names = [genre["name"] for genre in data["genres"]]
    vote_average = data["vote_average"]
    vote_count = data["vote_count"]
    cast = [actor["name"] for actor in data_credits["cast"][:5]]
    director = next((person["name"] for person in data_credits["crew"] if person["job"] == "Director"), "Unknown")
    duration = data["runtime"]  

    print(f"Title: {title}")
    print(f"Genres: {', '.join(genre_names)}")
    print(f"Vote average: {vote_average}")
    print(f"Vote count: {vote_count}")
    print(f"Director: {director}")
    print(f"Duration in minutes: {duration}")
    print(f"Main cast: {', '.join(cast)}")
    
    return title, genre_names, vote_average, vote_count, cast, director, duration

    # Print the extracted information

get_movie_details()


def movie_to_vector(id):
    title, genre_names, vote_average, vote_count, cast, director, duration = get_movie_details(id)
    data = {'directors': [1, 2], 'col2': [3, 4]}
    
    
def get_movie_list(genre_ids, min_rating, page=1, min_vote_ct=100):
    params = {"api_key": api_key, 
            "language": "en-US", 
            "page": "{}".format(page),
            "sort_by": "popularity.desc", 
            "with_genres": "{}".format(genre_ids),
            "vote_count.gte": "{}".format(min_vote_ct),
            "vote_average.gte": "{}".format(min_rating)}
    
    # Send a GET request to the API endpoint with the query parameters
    response = requests.get(discover_URL, params=params)
    data = json.loads(response.text)
    movies = data["results"]

    print("get_movie_list fetched: {} results:".format(len(movies)))
    # Print the list of movie titles
    for movie in movies[:5]:
        id = movie["id"]
        title = movie["title"]
        #print(title)
        #get_movie_details(id)
        #print("\n")

Title: Fight Club
Genres: Drama, Thriller, Comedy
Vote average: 8.434
Vote count: 26525
Director: David Fincher
Duration in minutes: 139
Main cast: Edward Norton, Brad Pitt, Helena Bonham Carter, Meat Loaf, Jared Leto


In [37]:
temp_person = Person("Peter", ["Action", "Thriller", "Science Fiction", "Adventure"])
temp_person.movies_to_ids(get_movie_ID=get_movie_ID)
for elem in temp_person.fav_movies:
    get_movie_details(elem)
    

Movie ID: 419704
Movie ID: 577922
Movie ID: 27205
Movie ID: 77
[419704, 577922, 27205, 77]
Title: Ad Astra
Genres: Science Fiction, Drama
Vote average: 6.099
Vote count: 5934
Director: James Gray
Duration in minutes: 123
Main cast: Brad Pitt, Tommy Lee Jones, Ruth Negga, John Ortiz, Liv Tyler
Title: Tenet
Genres: Action, Thriller, Science Fiction
Vote average: 7.204
Vote count: 8228
Director: Christopher Nolan
Duration in minutes: 150
Main cast: John David Washington, Robert Pattinson, Elizabeth Debicki, Kenneth Branagh, Dimple Kapadia
Title: Inception
Genres: Action, Science Fiction, Adventure
Vote average: 8.361
Vote count: 33560
Director: Christopher Nolan
Duration in minutes: 148
Main cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ken Watanabe, Tom Hardy, Elliot Page
Title: Memento
Genres: Mystery, Thriller
Vote average: 8.19
Vote count: 13226
Director: Christopher Nolan
Duration in minutes: 113
Main cast: Guy Pearce, Carrie-Anne Moss, Joe Pantoliano, Mark Boone Junior, Russ Fega


In [ ]:
# CREATE DATAFRAME OF USER PREFERENCES


In [47]:
relevant_ids = get_genre_ids(temp_person, get_genre_dict())  # FETCH IDs of specified genres

for i in range(1, 2):
    get_movie_list(genre_ids=relevant_ids, min_rating=5, page=i)

get_movie_list fetched: 20 results:
